In [1]:
from dotenv import load_dotenv
import os

load_dotenv()


LANGUAGE_KEY = os.getenv('LANGUAGE_KEY')
LANGUAGE_ENDPOINT = os.getenv('LANGUAGE_ENDPOINT')
LANGUAGE_ENDPOINT

'https://cw-language-summary-meetings.cognitiveservices.azure.com/'

In [ ]:
ls data

In [ ]:
import docx 


# path = "data/2024_11_22_Point avancement Digitalisation.docx"
doc = docx.Document('data/Appel avec Remi Cazelles-brut.txt')

In [ ]:
for i in range(2,10):
    print(doc.paragraphs[i].text)

Predefined Roles:

"Customer":
"Agent":

# define people

In [ ]:


people = ("list of names from the transcript file")
company = dict({"name1":{'id':1,
                                    'role':'Customer'},
                  "name2":{'id':2,
                                   'role':'Customer'},
                  "name3":{'id':3,
                                   'role':'Assistant'},
                  "name4":{'id':4,
                                    'role':'Customer'},                    
                  })


In [ ]:
doc.paragraphs[12].text

In [ ]:

# def assign_role_and_id(line):
#     nom = " ".join(line.split(" ")[:2])
#     participant_id = company[nom]["id"]
#     role = company[nom]["role"]
#     return participant_id, role



# formatted_paragraphs=list()

# for i, paragraph in enumerate(doc.paragraphs):
#     # Skip first empty paragraphs
#     if i >1:
    
#         if not len(paragraph.text.split("\n")) > 2:
#             continue
        
#         # print(paragraph.text)
        
#         text=list()
#         # Format each paragraph as a dictionary
#         for line in paragraph.text.split("\n"):
            
#             if line.startswith(people):  
#                 participant_id, role = assign_role_and_id(line)
                
#             elif line.startswith("Remi Cazelles"):    ### quel nom pour invité ou .. ?
#                 participant_id = 2
#                 role = "Cyber team"
#             else:
#                 if line:
#                     text.append(line.replace("\xa0",""))
                
#         formatted_paragraph = {
#             "text": ". ".join(text),
#             "id": str(i),  # Assign a unique ID based on the order
#             "role": role,      # Assign a role (adjust based on your document content)
#             "participantId": participant_id
#         }
        
#         # Append formatted paragraph to list
#         formatted_paragraphs.append(formatted_paragraph)

https://learn.microsoft.com/en-us/azure/ai-services/language-service/summarization/quickstart?tabs=conversation-summarization%2Clinux&pivots=programming-language-python

In [ ]:
formatted_paragraphs[11:17]

# format 2

In [ ]:
for i in range(4):
    print()
    print(doc.paragraphs[i].text)

In [ ]:

def assign_role_and_id(line):
    nom = " ".join(line.split(" ")[:2])
    participant_id = company[nom]["id"]
    role = company[nom]["role"]
    return participant_id, role

formatted_paragraphs=list()
characters_count=0
for i, paragraph in enumerate(doc.paragraphs):
    text=list()
    if i <= 2:
        continue
    # Format each paragraph as a dictionary
    for line in paragraph.text.split("\n"):
        
        if line == "":
            continue
        
        elif line.startswith(people):  
            participant_id, role = assign_role_and_id(line)
            

        else:
            if line:
                text.append(line.replace("\xa0",""))
                
                characters_count += len(line.replace("\xa0",""))
            
    formatted_paragraph = {
        "text": ". ".join(text),
        "id": str(i),  # Assign a unique ID based on the order
        "role": role,      # Assign a role (adjust based on your document content)
        "participantId": participant_id
    }
    
    # Append formatted paragraph to list
    formatted_paragraphs.append(formatted_paragraph)
    
print(characters_count,"characters")
print(characters_count/4,"tokens")

64258 characters
16064.5 tokens


# make chunks of conversation

In [21]:
import unicodedata

def get_text_length(text):
    """Calculate the length of a string in Unicode text elements."""
    return sum(1 for _ in unicodedata.normalize('NFC', text))

def chunk_text_by_length(conversation, max_length):
    """Split the conversation into chunks based on text length in Unicode elements."""
    chunks = []
    current_chunk = []
    current_length = 0

    for item in conversation:
        text_length = get_text_length(item['text'])
        if current_length + text_length <= max_length:
            current_chunk.append(item)
            current_length += text_length
        else:
            chunks.append(current_chunk)
            current_chunk = [item]
            current_length = text_length

    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

chunks = chunk_text_by_length(formatted_paragraphs,max_length=5200)

print(len(chunks))

14


All other preconfigured features (synchronous)	5,120 as measured by StringInfo.LengthInTextElements. If you need to submit larger documents, consider using the feature asynchronously.
All other preconfigured features (asynchronous)	125,000 characters across all submitted documents, as measured by StringInfo.LengthInTextElements (maximum of 25 documents).

# function to read output

In [17]:
def view_result(result):
    task_results = result["tasks"]["items"]
    for task in task_results:
        print(f"\n{task['taskName']} status: {task['status']}")
        task_result = task["results"]
        if task_result["errors"]:
            print("... errors occurred ...")
            for error in task_result["errors"]:
                print(error)
        else:
            conversation_result = task_result["conversations"][0]
            if conversation_result["warnings"]:
                print("... view warnings ...")
                for warning in conversation_result["warnings"]:
                    print(warning)
            else:
                summaries = conversation_result["summaries"]
                for summary in summaries:
                    print(f"{summary['aspect']}: {summary['text']}")

# appel API Azure

In [ ]:
# from azure.core.credentials import AzureKeyCredential
# from azure.ai.language.conversations import ConversationAnalysisClient



# conversation_id = 'start_meeting'#path.split("/")[1].split(".")[0]

# client = ConversationAnalysisClient(LANGUAGE_ENDPOINT, AzureKeyCredential(LANGUAGE_KEY))
# with client:
#     poller = client.begin_conversation_analysis(
#         task={
#             "displayName": "Analyze conversations from xxx",
#             "analysisInput": {
#                 "conversations": [
#                     {
#                         "conversationItems": formatted_paragraphs[:200],              # LIMIT HERE THE CONVERSATION LENGHT TO LIMIT COSTS
#                         "modality": "text",
#                         "id": conversation_id,
#                         "language": "fr",
#                     },
#                 ]
#             },
#             "tasks": [
#                 {
#                     "taskName": "Issue task",
#                     "kind": "ConversationalSummarizationTask",
#                     "parameters": {"summaryAspects": ["issue"]},
#                 },
#                 {
#                     "taskName": "Resolution task",
#                     "kind": "ConversationalSummarizationTask",
#                     "parameters": {"summaryAspects": ["resolution"]},
#                 },
#                     {
#         "taskName": "Recap task",
#         "kind": "ConversationalSummarizationTask",
#         "parameters": {"summaryAspects": ["recap"]},
#     },
#             ],
#         }
#     )
    
    

#     # view result
#     result = poller.result()
#     task_results = result["tasks"]["items"]
#     for task in task_results:
#         print(f"\n{task['taskName']} status: {task['status']}")
#         task_result = task["results"]
#         if task_result["errors"]:
#             print("... errors occurred ...")
#             for error in task_result["errors"]:
#                 print(error)
#         else:
#             conversation_result = task_result["conversations"][0]
#             if conversation_result["warnings"]:
#                 print("... view warnings ...")
#                 for warning in conversation_result["warnings"]:
#                     print(warning)
#             else:
#                 summaries = conversation_result["summaries"]
#                 for summary in summaries:
#                     print(f"{summary['aspect']}: {summary['text']}")

# appel API Azure chunks

In [ ]:
import unicodedata
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.conversations import ConversationAnalysisClient


# conversation_chunks = chunk_text_by_length(formatted_paragraphs,max_length=4500)

conversation_chunks = chunks

client = ConversationAnalysisClient(LANGUAGE_ENDPOINT, AzureKeyCredential(LANGUAGE_KEY))

try:
    results = []
    for idx, chunk in enumerate(conversation_chunks):
        conversation_id = f"meetingWenboRemi_part_{idx + 1}"
        poller = client.begin_conversation_analysis(
            task={
                "displayName": f"Analyze conversations part {idx + 1}",
                "analysisInput": {
                    "conversations": [
                        {
                            "conversationItems": chunk,
                            "modality": "text",
                            "id": conversation_id,
                            "language": "fr",
                        },
                    ]
                },
                "tasks": [
                    # {
                    #     "taskName": "Issue task",
                    #     "kind": "ConversationalSummarizationTask",
                    #     "parameters": {"summaryAspects": ["issue"]},
                    # },
                    # {
                    #     "taskName": "Resolution task",
                    #     "kind": "ConversationalSummarizationTask",
                    #     "parameters": {"summaryAspects": ["resolution"]},
                    # },
                    {
                        "taskName": "Recap task",
                        "kind": "ConversationalSummarizationTask",
                        "parameters": {"summaryAspects": ["recap"]},
                    },
                ],
            }
        )
        result = poller.result()
        results.append(result)
        view_result(result)
        
finally:
    client.close()

        

# Combine results as needed
for idx, result in enumerate(results):
    print(f"Results for part {idx + 1}: {result}")


In [ ]:
# view result
result = poller.result()

def view_result(result):
    task_results = result["tasks"]["items"]
    for task in task_results:
        print(f"\n{task['taskName']} status: {task['status']}")
        task_result = task["results"]
        if task_result["errors"]:
            print("... errors occurred ...")
            for error in task_result["errors"]:
                print(error)
        else:
            conversation_result = task_result["conversations"][0]
            if conversation_result["warnings"]:
                print("... view warnings ...")
                for warning in conversation_result["warnings"]:
                    print(warning)
            else:
                summaries = conversation_result["summaries"]
                for summary in summaries:
                    print(f"{summary['aspect']}: {summary['text']}")

# estimate token usage
"""
While not a precise way of tracking tokens, you can estimate token usage based on the size of the input text you're sending. The number of tokens depends on the service, but you can approximate that:

1 token is roughly 4 characters (or about 3/4 of a word).
For example, if you are sending a string of 400 characters, you can estimate that it uses around 100 tokens.
You can manually log the input size (character count) as a proxy for token usage, but the exact number of tokens can vary depending on the language model and other factors.
"""

## Track Token Usage via Azure Monitor

If you're looking for more detailed token usage statistics, Azure's Monitor and Cost Management tools are useful. Here's how you can programmatically track token usage:

Using Azure Monitor and Azure SDK for Python
Set up monitoring in Azure Monitor for the Cognitive Service resource.
Use Azure Monitor metrics to track the token usage for your Cognitive Services resource.
You can query metrics using the azure-mgmt-monitor SDK to get detailed data about token consumption (e.g., TotalTokens metric).


```from azure.identity import DefaultAzureCredential
from azure.mgmt.monitor import MonitorManagementClient
from azure.mgmt.monitor.models import MetricAggregationType

# Initialize the MonitorManagementClient
credential = DefaultAzureCredential()
monitor_client = MonitorManagementClient(credential, subscription_id)

# Specify the resource ID of the Cognitive Services
resource_id = "/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{account_name}"

# Get metrics for the cognitive service
metrics = monitor_client.metrics.list(
    resource_id,
    timespan="2024-11-01T00:00:00Z/2024-11-12T00:00:00Z",  # Specify your timespan
    metricnames="TotalTokens",  # Metric to track token usage
    aggregation=MetricAggregationType.SUM
)

for metric in metrics.value:
    print(f"Metric: {metric.name.localized_value}")
    for time_series in metric.timeseries:
        for data in time_series.data:
            print(f"Timestamp: {data.timestamp}, Total Tokens: {data.total}")```


## Check Azure Cost Management Programmatically

```from azure.mgmt.costmanagement import CostManagementClient
from azure.identity import DefaultAzureCredential

# Initialize the CostManagementClient
credential = DefaultAzureCredential()
cost_client = CostManagementClient(credential, subscription_id)

# Get cost data for a particular period
cost_data = cost_client.query.usage(
    scope="/subscriptions/{subscription_id}",
    time_period={"from": "2024-11-01", "to": "2024-11-12"},
    dataset={"granularity": "Daily"}
)

for entry in cost_data.value:
    print(f"Service: {entry['name']}, Cost: {entry['cost']}")```

# Asynchronous Calls

In [ ]:
azure.ai.language.conversations.aio.ConversationAnalysisClient

In [ ]:
import joblib

joblib.dump(task_result,"data/conv")

In [ ]:
import uuid

unique_conversation_id = uuid.uuid1()
str(unique_conversation_id)